In [2]:
import pandas as pd
import dask.dataframe as dd

In [78]:
# Define the data types for the columns with mismatched types
dtypes = {
    'Area Code': 'int64',
    'CBSA Code': 'float64',
    'CSA Code': 'float64',
    'Census Block': 'float64',
    'Census Tract': 'float64',
    'County Code': 'float64',
    'FIPS Code': 'float64',
    'Sales Volume (9) - Location': 'float64'
}

In [79]:
# Read the text file into a Dask DataFrame
ddf = dd.read_csv("2020_Business_Academic_QCQ.txt", encoding='ISO-8859-1', dtype=dtypes, assume_missing=True)

In [90]:
hilo_foot_traffic = pd.read_parquet('hilo_full_patterns.parquet')
hilo_foot_traffic['street_address']

2353                                300 W Lanikaula St
3153                          1263 Kilauea Ave Ste 320
3422                                   274 Kilauea Ave
4179                                        Piilani St
9436                                   655 Kilauea Ave
                              ...                     
506689                             190 Keawe St Ste 31
507478                                    126 Keawe St
514153    2450 Kekuanaoa St Hilo International Airport
534386                                 66 Kekuanaoa St
540183                                  190 Ululani St
Name: street_address, Length: 47954, dtype: object

In [81]:
name_visit_df = hilo_foot_traffic[['location_name', 'raw_visit_counts']]
# Drop rows with missing values in the name_visit_df DataFrame
name_visit_df_cleaned = name_visit_df.dropna()

In [82]:
# Group by company name and sum the visit counts
name_visit_agg_df = name_visit_df.groupby('location_name')['raw_visit_counts'].sum().reset_index().dropna()

In [83]:
# Filter the Dask DataFrame for the city of Hilo and the state of Hawaii
ddf_filtered = ddf[(ddf['City'] == 'HILO') & (ddf['State'] == 'HI')]

In [84]:
# Convert the aggregated pandas DataFrame to a Dask DataFrame
name_visit_agg_ddf = dd.from_pandas(name_visit_agg_df, npartitions=1)

# Merge the filtered Dask DataFrame with the aggregated Dask DataFrame on the company name
merged_ddf = dd.merge(ddf_filtered, name_visit_agg_ddf, left_on='Company', right_on='location_name', how='left')

In [85]:
# Merge the aggregated Dask DataFrame with the filtered pandas DataFrame on the company name
merged_df = name_visit_agg_ddf.merge(ddf_filtered, left_on='location_name', right_on='Company', how='left')

In [86]:
# Select only the necessary columns from the merged Dask DataFrame
merged_df = merged_df[['Company', 'raw_visit_counts', 'Sales Volume (9) - Location']]

In [88]:
# Replace NA values with -1 in the integer column
merged_df['Sales Volume (9) - Location'] = merged_df['Sales Volume (9) - Location'].fillna(0)

In [96]:
merged_df = merged_df.dropna()

ValueError: Metadata inference failed in `dropna`.

You have supplied a custom function and Dask is unable to 
determine the type of output that that function returns. 

To resolve this please provide a meta= keyword.
The docstring of the Dask function you ran should have more information.

Original error is below:
------------------------
TypeError('You cannot set both the how and thresh arguments at the same time.')

Traceback:
---------
  File "/opt/anaconda3/lib/python3.8/site-packages/dask/dataframe/utils.py", line 175, in raise_on_meta_error
    yield
  File "/opt/anaconda3/lib/python3.8/site-packages/dask/dataframe/core.py", line 5510, in _emulate
    return func(*_extract_meta(args, True), **_extract_meta(kwargs, True))
  File "/opt/anaconda3/lib/python3.8/site-packages/dask/utils.py", line 900, in __call__
    return getattr(obj, self.method)(*args, **kwargs)
  File "/opt/anaconda3/lib/python3.8/site-packages/pandas/util/_decorators.py", line 331, in wrapper
    return func(*a

In [95]:
# Compute the correlation between visit counts and sales
correlation_result = merged_df['raw_visit_counts'].corr(merged_df['Sales Volume (9) - Location']).compute()

/opt/anaconda3/lib/python3.8/site-packages/dask/dataframe/multi.py:1217: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indices of each dataframes are 
 aligned. This assumption is not generally safe.
  warnings.warn(


ValueError: Integer column has NA values in column 7

In [74]:
# Check for NA values in the integer column after filling with zeros
na_count = (ddf['Sales Volume (9) - Location'] == 0).sum().compute()

ValueError: Integer column has NA values in column 7